In [1]:
import pandas as pd
import numpy as np
import requests
pd.options.display.max_columns = None

In [8]:
def comprobar_duplicados(input_file):
    # Leer el archivo CSV
    df = pd.read_csv(input_file, header=None, names=['URL'])
    
    # Mostrar el número total de enlaces y enlaces únicos
    total = len(df)
    unique = df['URL'].nunique()
    duplicates = total - unique
    print(f"Total de enlaces: {total}")
    print(f"Enlaces únicos: {unique}")
    print(f"Enlaces duplicados: {duplicates}")
    
    # Opcional: Mostrar algunos enlaces duplicados
    duplicated_links = df[df.duplicated('URL', keep=False)]
    if not duplicated_links.empty:
        print("\nAlgunos enlaces duplicados:")
        print(duplicated_links)
    else:
        print("\nNo se encontraron enlaces duplicados exactos.")

In [10]:
comprobar_duplicados('link_casas_limpios.csv')

Total de enlaces: 78104
Enlaces únicos: 58503
Enlaces duplicados: 19601

Algunos enlaces duplicados:
                                                     URL
0      https://www.portalinmobiliario.com/MLC-2616734702
1      https://www.portalinmobiliario.com/MLC-1492785093
2      https://www.portalinmobiliario.com/MLC-1520347535
3      https://www.portalinmobiliario.com/MLC-2703866948
4      https://www.portalinmobiliario.com/MLC-2216057850
...                                                  ...
76739  https://www.portalinmobiliario.com/MLC-1510712137
76740  https://www.portalinmobiliario.com/MLC-1485024693
77379  https://www.portalinmobiliario.com/MLC-1526781031
77396  https://www.portalinmobiliario.com/MLC-2463747180
77459  https://www.portalinmobiliario.com/MLC-2679137770

[33569 rows x 1 columns]


In [12]:
comprobar_duplicados('link_deptos_limpios.csv')

Total de enlaces: 75296
Enlaces únicos: 60017
Enlaces duplicados: 15279

Algunos enlaces duplicados:
                                                     URL
0      https://www.portalinmobiliario.com/MLC-2709511016
1      https://www.portalinmobiliario.com/MLC-2716491448
2      https://www.portalinmobiliario.com/MLC-1526984223
3      https://www.portalinmobiliario.com/MLC-2707975422
4      https://www.portalinmobiliario.com/MLC-1509387077
...                                                  ...
75291  https://www.portalinmobiliario.com/MLC-2700953240
75292  https://www.portalinmobiliario.com/MLC-2705537730
75293  https://www.portalinmobiliario.com/MLC-2707322284
75294  https://www.portalinmobiliario.com/MLC-2712215406
75295  https://www.portalinmobiliario.com/MLC-2713644568

[26614 rows x 1 columns]


Debería haber al rededor de 13.000 duplicados en casas y al rededor de 2.300 en departamentos.

In [2]:
df_deptos = pd.read_csv('dataset_deptos.csv')
df_deptos.sample(5)

,id,link,nombre,fecha_consultado,fecha_publicado,divisa,monto,ppm2,ppm2_avg,superficie_t,...,bodegas,cantidad_pisos,dep_por_piso,piso,tipo_depto,orientacion,antiguedad,gastos_comunes,latitud,longitud
14671,2692679290,https://portalinmobiliario.com/MLC-2692679290,Departamento En Venta De 3 Dorm. En La Reina,2025-04-05,NaN,UF,14.499,96 UF/m²,91 UF/m²,190 m²,...,1.0,4.0,2.0,NaN,Departamento,P,3 años,NaN,-33.442797,-70.519098
6401,1568646125,https://portalinmobiliario.com/MLC-1568646125,Departamento Avenida Inglaterra Id: 127125,2025-04-21,Publicado hace 3 meses por Houm,UF,1.300,43 UF/m²,62 UF/m²,30 m²,...,NaN,NaN,NaN,24.0,Departamento,P,9 años,NaN,-33.412480,-70.655121
4283,1580048945,https://portalinmobiliario.com/MLC-1580048945,Departamento En Venta En Quilicura,2025-04-20,Publicado hace 55 días,$,47.000.000,26 UF/m²,46 UF/m²,45 m²,...,NaN,3.0,NaN,3.0,Departamento,NP,0 años,NaN,-33.372134,-70.721414
590,2817681394,https://portalinmobiliario.com/MLC-2817681394,Departamento Camino Los Ingleses Id: 134135,2025-04-13,Publicado hace 3 meses por Houm,UF,3.050,46 UF/m²,50 UF/m²,66 m²,...,NaN,NaN,NaN,NaN,Departamento,S,0 años,NaN,-33.055192,-71.589316
1990,2862163130,https://portalinmobiliario.com/MLC-2862163130,"Espacioso Depto + Azotea Privada. El Tranque, ...",2025-04-19,Publicado hace 24 días,UF,19.000,102 UF/m²,97 UF/m²,247 m²,...,1.0,NaN,NaN,NaN,NaN,N,17 años,NaN,-33.357149,-70.524586


In [ ]:

# Valor actual de la UF. 24 abril 2025
valor_uf = 39036.45

# Paso 1: Filtrar filas con divisa en "$"
mask_pesos = df_deptos['divisa'] == "$"

# Paso 2: Limpiar la columna 'monto' removiendo puntos y convirtiendo a float
df_deptos.loc[mask_pesos, 'monto'] = (
    df_deptos.loc[mask_pesos, 'monto']
    .str.replace('.', '', regex=False)  # Eliminar separador de miles
    .str.replace(',', '.', regex=False)  # Por si hay coma decimal
    .astype(float)
)

# Paso 3: Convertir los montos a UF
df_deptos.loc[mask_pesos, 'monto'] = df_deptos.loc[mask_pesos, 'monto'] / valor_uf

# Paso 4: Cambiar la divisa a "UF"
df_deptos.loc[mask_pesos, 'divisa'] = "UF"


In [8]:
# Lista de columnas y sus patrones a remover
columnas_y_unidades = {
    'monto': '',            # Ya lo tratamos antes, solo asegúrate que esté en float
    'ppm2': ' UF/m²',
    'ppm2_avg': ' UF/m²',
    'superficie_t': ' m²',
    'superficie_u': ' m²',
    'terraza': ' m²'
}

# Limpiar y convertir a float
for col, unidad in columnas_y_unidades.items():
    df_deptos[col] = (
        df_deptos[col]
        .astype(str)                          # Asegurar que sean strings
        .str.replace('.', '', regex=False)    # Eliminar separador de miles
        .str.replace(',', '.', regex=False)   # Reemplazar coma por punto decimal
        .str.replace(unidad, '', regex=False) # Eliminar unidad
        .str.strip()                          # Eliminar espacios al inicio/fin
        .replace('', '0')                     # Por si quedó vacío
        .astype(float)
    )


In [9]:
df_deptos.sample(5)

,id,link,nombre,fecha_consultado,fecha_publicado,divisa,monto,ppm2,ppm2_avg,superficie_t,superficie_u,terraza,ambientes,dormitorios,banos,ubicacion,estacionamiento,bodegas,cantidad_pisos,dep_por_piso,piso,tipo_depto,orientacion,antiguedad,gastos_comunes,latitud,longitud
6856,1589117617,https://portalinmobiliario.com/MLC-1589117617,Departamento Avenida Edmundo Eluchans Id: 142760,2025-04-23,Publicado hace 33 días por Houm,UF,6.200000e+03,78.0,79.0,91.0,79.0,NaN,2.0,2.0,2.0,"Avenida Edmundo Eluchans, Reñaca, Viña Del Mar...",1.0,1.0,NaN,NaN,NaN,Departamento,SP,6 años,NaN,-32.956180,-71.543516
9705,2852109044,https://portalinmobiliario.com/MLC-2852109044,Departamento 2 D 1 B Con Excelente Conectividad.,2025-04-04,NaN,UF,2.305538e+15,41.0,53.0,55.0,55.0,NaN,6.0,2.0,2.0,"Mapocho, 1450, Centro Histórico de Santiago, S...",NaN,NaN,NaN,NaN,8.0,Departamento,S,14 años,NaN,-33.432090,-70.658333
1406,2860933600,https://portalinmobiliario.com/MLC-2860933600,Venta Departamento 2h 2b Con Bodega,2025-04-16,Publicado hace 25 días,UF,2.350000e+03,61.0,72.0,42.0,38.0,2.0,3.0,2.0,2.0,"Lira, 300, Santa Isabel, Santiago, RM (Metropo...",NaN,1.0,22.0,12.0,15.0,Departamento,N,14 años,NaN,-33.448047,-70.639550
14820,2807860736,https://portalinmobiliario.com/MLC-2807860736,3 Dormitorios-metro Los Dominicos-las Condes-p...,2025-04-05,NaN,UF,5.900000e+03,75.0,107.0,82.0,78.0,4.0,NaN,3.0,2.0,"Chesterton 7168, 7571879 Las Condes, Región Me...",NaN,NaN,NaN,NaN,1.0,Departamento,N,43 años,NaN,-33.406984,-70.551191
7756,2776633908,https://portalinmobiliario.com/MLC-2776633908,Venta Dpto Con Linda Vista Al Mar En Marbella,2025-04-23,Publicado hace 5 meses por Engel & Völkers Costa,UF,1.399000e+04,127.0,116.0,124.0,110.0,14.0,4.0,4.0,3.0,"Vía Sin Nombre, Marbella, Puchuncaví, Valparaíso",1.0,1.0,1.0,3.0,2.0,Departamento,SP,14 años,NaN,-32.664440,-71.431046


In [10]:
df_deptos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19463 entries, 0 to 19462
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                19463 non-null  int64  
 1   link              19463 non-null  object 
 2   nombre            19463 non-null  object 
 3   fecha_consultado  19463 non-null  object 
 4   fecha_publicado   14290 non-null  object 
 5   divisa            19463 non-null  object 
 6   monto             19463 non-null  float64
 7   ppm2              18937 non-null  float64
 8   ppm2_avg          18874 non-null  float64
 9   superficie_t      19463 non-null  float64
 10  superficie_u      19226 non-null  float64
 11  terraza           11835 non-null  float64
 12  ambientes         6457 non-null   float64
 13  dormitorios       18952 non-null  float64
 14  banos             17798 non-null  float64
 15  ubicacion         19463 non-null  object 
 16  estacionamiento   11784 non-null  float6

In [11]:
df_deptos.to_csv('dataset_deptos.csv', index=False, encoding='utf-8')

In [5]:
import numpy as np
import pandas as pd

# Cargar tu dataset de departamentos
df_deptos = pd.read_csv("Datasets/deptos.csv")  # ajusta ruta y nombre

# Coordenadas de tu universidad -33.4347665,-70.614827
uni_lat = -33.4347665
uni_lon = -70.614827

# Función Haversine para distancia en metros
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radio de la Tierra en metros
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

# Calcular la distancia desde cada depto a la universidad
df_deptos["dist_uni_m"] = haversine(
    uni_lat, uni_lon,
    df_deptos["latitud"].values,
    df_deptos["longitud"].values
)

# Obtener el departamento más cercano
idx_min = df_deptos["dist_uni_m"].idxmin()
depto_cercano = df_deptos.loc[idx_min]

print("ID depto más cercano:", depto_cercano["id"])
print("Distancia a la universidad (m):", depto_cercano["dist_uni_m"])


ID depto más cercano: 1591686159
Distancia a la universidad (m): 44.923871288905666


C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_33440\3157632602.py:5: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_deptos = pd.read_csv("Datasets/deptos.csv")  # ajusta ruta y nombre
